In [ ]:
# xclara: C = 3, |D| = 3000, F = 2
# s-set1: C = 15, |D| = 5000, F = 2
# pendigits: C = 10, |D| = 10992, F = 16
# waveform: C = 3, |D| = 5000, F = 21

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import itertools
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment as linear_assignment

ds = 'xclara' #'xclara','s-set1','pendigits','waveform'
print(ds)
df = pd.read_csv(f'dataset/{ds}.csv')
display(df.head(2))
print('n_cluster',len(df.iloc[:,-1].value_counts()))
print('n_feature',df.shape[1]-1)
N_clusters = len(df.iloc[:,-1].value_counts())
N_features = df.shape[1]-1

# **Inserisco i vari parametri**

In [ ]:
# N_clusters: int = 3
# N_features:int = 2
random_state = centers_seed = random_clients_seed = iid_seed = shuffle_seed = 2 #123
factor_lambda:float = 2
epsilon:float = 0.005
max_number_rounds:int = 30
mode = 'iid' # iid  |  non_iid_volume  |  non_iid_distr

# **Definisco le funzioni per le nrme e inizializzo i centroidi**

In [ ]:
from abc import ABC, abstractmethod
from typing import Callable, Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
from scipy.io import arff
import math
from random import randint
from numba import jit

from sklearn.metrics import adjusted_rand_score as ARI
FROBENIUS_NORM = 'fro'


#@jit(nopython=True)
#def numba_norm(u:np.ndarray, v:np.ndarray):
#    return np.linalg.norm(u - v)

def norm_fro(u:np.ndarray):
    return np.linalg.norm(u, ord = FROBENIUS_NORM)

# hungarian method (https://smorbieu.gitlab.io/accuracy-from-classification-to-clustering-evaluation/)
def perm_norm_fro(u:np.ndarray, v:np.ndarray):
    cm = cdist(u, v)
#     print(cm)
    indexes = linear_assignment(cm)
    indexes = np.transpose(indexes)
    js = [e[1] for e in sorted(indexes, key=lambda x: x[0])]
#     print(js)
    v = np.asarray(v)
    original_norm = np.linalg.norm(u-v, ord = FROBENIUS_NORM)
    permuted_norm = np.linalg.norm(u-v[js], ord = FROBENIUS_NORM)
#     print('original centers: ',v)
#     print('permuted centers: ',v[js])
#     print('original norm: ',original_norm)
#     print('permuted norm: ',permuted_norm)
    print()
    return permuted_norm


def generate_random_centers(seed: int, n_clusters: int, n_features: int):
    import numpy as np
    np.random.seed(seed)
    centers = np.random.rand(n_clusters,n_features)
    return centers

# **FederatedFCMeans Client**

In [ ]:
class FCMeansFederatedClient:

    def initialize(self, params: Dict) -> None:
        self.__dataset = params['dataset']
        self.__num_features = len(self.__dataset[0])
        self.__classes = [-1] * len(self.__dataset)
        self.__distance_fn = params['distance_fn']
        self.__factor_lambda = params['factor_lambda']
        self.y_true = params['y_true']
        
    def evaluate_cluster_assignment(self, centers: List) -> List[Tuple]:
        return self.__local_sums(centers)

    def finalize(self, centers: List[np.array]) -> None:
        

    def __local_sums(self, centers: List[np.array]):
        


# **FederatedFCMeans Server**

In [ ]:
class FCMeansFederatedServer:

    def __init__(self):
        self.__current_round = 0

    def initialize(self, params: Dict) -> None:
        self.__epsilon = params['epsilon']
        self.__max_number_rounds = params.get('max_number_rounds', 10)
        self.__num_clusters = params['num_clusters']
        self.__num_features = params['num_features']
        self.__norm_fm = params['norm_fn']
        self.__fnorms = []
        self.__cluster_centers = []
        self.__cluster_centers.append(params['centers'])

    def next_round(self) -> bool:
        num_clusters = self.__num_clusters
        cluster_centers = self.__cluster_centers
        
        num_centers = len(cluster_centers)
        if (num_centers > 1):
            centers_r = np.array(cluster_centers[-1])
            centers_r_1 = np.array(cluster_centers[-2])
            tot_diff_sum = self.__norm_fm(centers_r- centers_r_1)
            self.__fnorms.append(tot_diff_sum)
#             if (tot_diff_sum < self.__epsilon):
#                 return False
        
        result = self.__current_round < self.__max_number_rounds
        
        self.__current_round = self.__current_round + 1 if result else 0
        return result

    def process_clustering_results(self, client_responses: List):
        """
        num_clients = len(client_responses)
        num_clusters = self.__num_clusters

        #client_responses is a list of list of tuples where the first element of this tuple is u and second element is ws
        num_features = self.__num_features
        u_list = [0] * num_clusters
        ws_list = [[0] * num_features for i in range(num_clusters)]
        
        for client_idx in range(num_clients):
            # remember the response is a list of tuples where each tuple represents the (u, ws) for each cluster
            response = client_responses[client_idx]
            for i in range(num_clusters):
                client_u = response[0][i]
                client_ws = response[1][i] if response[1][i] is np.array else np.array(response[1][i])
                u_list[i] = u_list[i] + client_u
                ws_list[i] = ws_list[i] + client_ws

        new_cluster_centers = []
        prev_cluster_centers = self.__cluster_centers[-1]
        
        for i in range(num_clusters):
            u = u_list[i]
            ws = ws_list[i]
            if (u == 0):
                center = prev_cluster_centers[i]
            else:
                center = ws/u
            new_cluster_centers.append(np.array(center))

        self.__cluster_centers.append(new_cluster_centers)
        """
        
    def get_centers(self) -> List:
        return self.__cluster_centers[-1]

    #@property
    def current_round(self) -> int:
        return self.__current_round

    def finalize(self, enabled_print: bool = False) -> None:
        """ HERE WE CAN SAVE FOR EACH ROUND THE VALUES OF THE CENTERS """
        centers = self.__cluster_centers[1:]
        fnorms = self.__fnorms
        
        
        if (enabled_print):
            if N_features ==2:
                for i in range(len(centers)):
                    plt.figure()
                    plt.title(f"round {i}, Frobenius norm {fnorms[i]:.15f}")
                    plt.scatter(X[:,0],X[:,1])
                    plt.scatter(np.asarray(centers[i])[:,0],np.asarray(centers[i])[:,1],s = 30)
        return (fnorms, centers)

# **Genero i chunks del dataset (prima bilanciati e poi, eventualmente, sbilanciati)**

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle as shuffleSK

#UPDATE: ADDED iid_seed and shuffle_seed parameters

def generate_dataset_chunks(X: np.array, 
                            Y: np.array, 
                            n_splits: int, 
                            shuffle: bool = False, 
                            shuffle_seed:int = None, 
                            mode: str = 'iid', 
                            iid_seed: int = None):
    if (n_splits == 1):
        return [X],[Y],[]
    dataset_chunks = []
    y_chunks = []
    all_indices = []

    if mode == 'iid':
        skf = StratifiedKFold(n_splits = n_splits, shuffle = shuffle, random_state = iid_seed)
        for train_index, test_index in skf.split(X, Y):
            dataset_chunks.append(X[test_index])
            y_chunks.append(Y[test_index])
            all_indices.extend(test_index)

    """
    elif mode == 'non_iid_volume':
        X, Y = shuffleSK(X,Y, random_state = shuffle_seed)

        factor = 1.2
        total = len(X)-int(np.floor(len(X)/factor))
        print(total)
        temp = []
        for i in range(n_splits-1):
            val = np.random.randint(0, total)
            temp.append(val)
            total -= val
        temp.append(total)
        nums = [z+int(np.floor(len(X)/factor/n_splits)) for z in temp]
        cumsum = list(np.cumsum(nums))
        old = [x for x in cumsum]
        cumsum.pop(-1)
        cumsum.insert(0,0)
        for s,e in zip(cumsum,old):
            dataset_chunks.append(X[s:e])
            # not tested
            y_chunks.append(Y[s:e])
    elif mode == 'non_iid_distr':
        dataset_chunks = [X[i:i + int(np.floor(len(X)/n_splits)),:] for i in range(0, len(X), int(np.floor(len(X)/n_splits)))]
        # not tested
        y_chunks.append([Y[i:i + int(np.floor(len(X)/n_splits)),:] for i in range(0, len(X), int(np.floor(len(X)/n_splits)))])
    return dataset_chunks,y_chunks, all_indices
    """

# **Dataset Initialization**

In [ ]:
X_original = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values
df.iloc[:,-1].value_counts()

# **Plot dei cluster (con 2 features)**

In [ ]:
if N_features == 2:
    plt.scatter(X_original[:,0], X_original[:,1],c = Y)

# **Plot dei cluster normalizzato (con 2 features)**

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X_original)
# X = X_original
if N_features == 2:
    plt.scatter(X[:,0],X[:,1], c = Y)

# **Execution Logic**

In [ ]:
def run_experiment(N_clients: int, 
                   N_min_clients: int, 
                   centers: List[np.array], 
                   dataset_chunks: List[np.array], 
                   y_chunks: List, 
                   client_parms: Dict[str, bytes],
                   server_params: Dict[str, bytes], 
                   enabled_print: bool = False, 
                   random_clients: bool = True,
                   random_clients_seed: int = None):
    #UPDATE: SEED INITIALIZATION MOVED HERE
    import random
    random.seed(random_clients_seed)

    # (1) CLIENTS INITIALIZATION ==================================================================
    clients = []

    for cli in range(N_clients):
        
        client_dataset = dataset_chunks[cli]
        params = dict(client_parms)
        client_y = y_chunks[cli]
        params['dataset'] = client_dataset
        params['y_true'] = client_y

        client = FCMeansFederatedClient()
        client.initialize(params)
        clients.append(client)

    print(f'starting server, total number of clients {N_clients}, min number of clients {N_min_clients}')

    active_clients_idx: List = None
    all_clients = (N_clients == N_min_clients)

    if (not random_clients and not all_clients):
        active_clients_idx = random.sample(range(N_clients), N_min_clients)


    # (2) SERVER INITIALIZATION ==================================================================
    server = FCMeansFederatedServer()
    server.initialize(server_params)

    # (3) FEDERATED TRAINING ====================================================================
    while (server.next_round()):

        if (random_clients and not all_clients):
            active_clients_idx = random.sample(range(N_clients), N_min_clients)
        
        centers = server.get_centers()
        client_responses = []
        client_responses_append = client_responses.append
        for cli, client in enumerate(clients):
            if (not all_clients and cli not in active_clients_idx):
                continue
            response = client.evaluate_cluster_assignment(centers)
            client_responses_append(response)

        server.process_clustering_results(client_responses)
    
    centers = server.get_centers()
    # (4) EXECUTING SOME FINAL LOGIC ==================================================================
    ari_list = []
    y_true_list = []
    y_pred_list = []
    for client in clients:
        u_matrix,y_pred = client.finalize(centers)
        y_true = client.y_true
        ari_list.append(ARI(y_true,y_pred))
        y_pred_list.extend(y_pred)
        y_true_list.extend(y_true)
#     print(u_matrix,y_pred)
    ari_mean = np.mean(np.asarray(ari_list))
    ari_global = ARI(y_true_list,y_pred_list)

    print('\tmedia: ',ari_mean)
    print('\tglobal:',ari_global)

    return server.finalize(enabled_print), ari_mean,ari_global, y_pred_list

# **Plots Definitions**

## **Plot che mostra dopo quante iterazioni raggiungiamo la convergenza**

In [ ]:
def fn_fnorm_by_round_experiment_plot(all_forms_list, epsilon, dataset_file,N_min_clients_percents,rnd = True):
    fig, ax = plt.subplots(figsize = (9,4))

    marker = itertools.cycle(('d', 's', 'o', '*','<','>','+','x')) 
    labels = [f'{label:.2f}' for label in N_min_clients_percents]
    
    
    
    val = np.nanmean(all_forms_list,axis = 0)
    std = np.nanstd(all_forms_list,axis = 0)
    for i in range(len(val)):
        Y_values = val[i]
        X_labels = [i for i in range(1, len(Y_values) + 1)]
        plt.plot(X_labels, Y_values, label = labels[i], marker = next(marker),linestyle = '--')
        Y_std = std[i]
        plt.fill_between(X_labels, Y_values-Y_std, Y_values+Y_std, alpha = 0.2)

    
#     plt.ylim([-0.01,1])
    plt.ylim([-0.01,0.1])

    #threshold line
    X_axis_threshold = [i for i in range(1, max_number_rounds + 1)]
    Y_axis_threshold = [epsilon] * max_number_rounds
    plt.text(1,epsilon+0.003,r'$\varepsilon$',size = 14)
    plt.plot(X_axis_threshold, Y_axis_threshold , 'k--', alpha=0.9)
    plt.xticks(range(1,max_number_rounds+1,2))
    plt.grid()

    # Labeling the X-axis 
    plt.xlabel('t (round)',size = 14) 
    plt.ylabel(r'$||V^{(t+1)} - V^{(t)}||_F$',size = 14) 
    legend = plt.legend(title=r"$\gamma$",fontsize = 14)
    plt.setp(legend.get_title(),fontsize=14)
    plt.tight_layout()
    plt.savefig('results/'+ds+'_round'+'_random'*rnd+'.pdf',format='pdf',bbox_inches='tight')

    plt.show()

## **Plot che mostra la proporzione dei partecipanti rispetto all'errore (0.25, 0.50, 0.75, 1)**

In [ ]:
def fn_fnorm_by_experiment_plot(N_min_clients_percents_param, fro_list_param,rnd):
    plt.figure(figsize=(4, 4))

    X_labels = [f'{per:.2f}' for per in N_min_clients_percents_param]
    Y_values = fro_list_param.mean(axis = 0)
    Y_std = fro_list_param.std(axis = 0)
    
    plt.plot(X_labels, Y_values,'ko-')
    plt.fill_between(X_labels, Y_values-Y_std, Y_values+Y_std, color = 'k',alpha = 0.1)
    plt.grid()
#     plt.ylim([-0.0005,0.016])

    # Labeling the X-axis 
    plt.xlabel(r"Fraction of participants $\gamma$", size = 14) 
    plt.ylabel(r'$||V_{fed}(\gamma) - V_{sum}||_F$',size = 14) 
#     plt.ylabel(r'$Fro(V_{federated} - V_{centralized})$') 
#     plt.title(f'Variation of Frobenius norm value for each % participant experiment. Dataset: {dataset_file}.',fontweight="bold") 
    plt.tight_layout()
    plt.savefig('results/'+ds+'_centr'+'_random'*rnd+'.pdf',format='pdf',bbox_inches='tight')
    plt.show()

# **Esecuzione dell'esperimento**

## **Definisco la funzione per eseguire l'algoritmo**

In [ ]:
#DISTRIBUITED APPROACH
def df_complete_experiment(random_clients:bool = False, 
                           centers_seed: int = None,
                           random_clients_seed: int = None, 
                           num_splits:int = 10):
    centers = generate_random_centers(centers_seed, N_clusters, N_features)

    N_clients:int = 20
    shuffle_dataset:bool = True
    dataset_chunks,y_chunks,all_indices = generate_dataset_chunks(X, Y, N_clients, shuffle = shuffle_dataset, mode = mode, shuffle_seed = shuffle_seed, iid_seed = iid_seed)

    enabled_print:bool = False

    client_params: Dict[str, bytes] = {
        'distance_fn': numba_norm,
        'factor_lambda': factor_lambda
    }

    server_params: Dict[str, bytes] = {
        'epsilon': epsilon,
        'max_number_rounds': max_number_rounds,
        'num_clusters': N_clusters,
        'num_features': N_features,
        'norm_fn': norm_fro,
        'centers': centers
    }

    ### DISTRIBUITED APPROACH, GOES FROM 0.25 to 1.0 percentage of participants from a total of N_clients available participants
    N_min_clients_percents: List[float] = [float(1/num_splits* i)  for i in range(1, num_splits + 1)]
    print(N_min_clients_percents)
    centers_list : List[List] = []
    all_forms_list: List[List] = []
    ari_clients_list: List[List] = []
    ari_clients_global: List[List] = []
    all_y_pred_fed: List[List] = []
    all_ari_fed_sum: List[List] = []
    for i in range(num_splits):
        per_clients = N_min_clients_percents[i]
        N_min_clients:int = math.floor(N_clients * per_clients)
        (fnorms, centers),ari_clients_mean,ari_global,y_pred_fed = run_experiment(N_clients, N_min_clients, 
                                                                                  centers, dataset_chunks, 
                                                                                  y_chunks, client_params, 
                                                                                  server_params, enabled_print,
                                                                                  random_clients, 
                                                                                  random_clients_seed = random_clients_seed)
        ari_clients_list.append(ari_clients_mean)
        ari_clients_global.append(ari_global)
        all_forms_list.append(fnorms)
        all_y_pred_fed.append(np.asarray(y_pred_fed))
        centers_list.append(centers[-1])

    ### CENTRALIZED APPROACH
    N_clients:int = 1
    shuffle_dataset:bool = True
    dataset_chunks,y_chunks,_ = generate_dataset_chunks(X, Y, N_clients, shuffle = shuffle_dataset)
    N_min_clients:int = math.floor(N_clients * per_clients)
    (fnorms_centralized, centers_centralized),_,_,y_pred_sum = run_experiment(N_clients, N_min_clients, centers, dataset_chunks,y_chunks, client_params, server_params, enabled_print,
                                                             random_clients, random_clients_seed = random_clients_seed)

    centralized_app_center = np.array(centers_centralized[-1])
    for gamma_pred_fed in all_y_pred_fed:
        all_ari_fed_sum.append(ARI(gamma_pred_fed,np.asarray(y_pred_sum)[all_indices]))
    return (centers_list, 
            all_forms_list, 
            centralized_app_center, 
            fnorms_centralized,
            N_min_clients_percents,
            ari_clients_list,
            ari_clients_global,
            all_ari_fed_sum)

In [ ]:
num_splits = 4
random_clients = True

fro_list_all = []
all_forms_list_all = []
ari_clients_list_all = []
ari_clients_global_all = []
ari_pred_fed_sum_all = []
for rep in range(10):
    print(rep,'seed')
    centers_list, all_forms_list, centralized_app_center, fnorms_centralized, N_min_clients_percents,ari_clients_list,ari_clients_global,all_ari_fed_sum = df_complete_experiment(
        random_clients,
        rep, # vario i centri 
        rep, # vario i client che seleziono
        num_splits)
    
    fro_list: List[float] = []
    for i in range(num_splits):
        center_i = centers_list[i]
        #fro_list.append(norm_fro(centralized_app_center - center_i)) # originale
        fro_list.append(perm_norm_fro(centralized_app_center, center_i))
    fro_list_all.append(fro_list)
    for l in all_forms_list:
        l += [np.nan] * (max_number_rounds - len(l))
    all_forms_list_all.append(all_forms_list)
    ari_clients_list_all.append(ari_clients_list)
    ari_clients_global_all.append(ari_clients_global)
    ari_pred_fed_sum_all.append(all_ari_fed_sum)
    print()
fro = np.asarray(fro_list_all)
all_fnorms = np.asarray(all_forms_list_all)
ari_clients = np.asarray(ari_clients_list_all)
ari_global = np.asarray(ari_clients_global_all)
ari_fed_sum = np.asarray(ari_pred_fed_sum_all)
ari_fed_sum.shape, ari_global.shape

# **Plots dei risultati**

## **Tabella ARI a seconda della percentuale dei partecipanti**

In [ ]:
metrics = pd.DataFrame()
metrics['ari_global_mean'] = ari_global.mean(axis = 0)
metrics['ari_global_std'] = ari_global.std(axis = 0)
metrics['ari_clients_mean'] = ari_clients.mean(axis = 0)
metrics['ari_clients_std'] = ari_clients.std(axis = 0)
metrics['fro_centers_mean'] = fro.mean(axis = 0)
metrics['fro_centers_std'] = fro.std(axis = 0)
metrics['ari_pred_fed_mean'] = ari_fed_sum.mean(axis = 0)
metrics['ari_pred_fed_std'] = ari_fed_sum.std(axis = 0)

metrics = metrics.round(5)
metrics.index = [f'{label:.2f}' for label in N_min_clients_percents]
metrics
metrics.to_csv(f'results/{ds}_metrics.csv')
pd.read_csv(f'results/{ds}_metrics.csv', index_col = 0)

# **Plot che mostra la proporzione dei partecipanti rispetto all'errore (0.25, 0.50, 0.75, 1) **

In [ ]:
fn_fnorm_by_experiment_plot(N_min_clients_percents, fro,rnd = True)

# **Plot che mostra dopo quante iterazioni raggiungiamo la convergenza**

In [ ]:
fn_fnorm_by_round_experiment_plot(all_fnorms, epsilon, ds, N_min_clients_percents,rnd = True)

In [ ]:
centers_list